In [ ]:
# import sys
# !{sys.executable} -m pip install psycopg2-binary jupyterlab-plotly

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as psql
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
conn = psycopg2.connect(host='telegram', dbname='telegram', user='postgres', password='root')
df = psql.read_sql_query('select * from "messages"',con=conn)
channels = psql.read_sql_query('select * from "channels"',con=conn)
users = psql.read_sql_query('select * from "users"',con=conn)

/tmp/ipykernel_3838726/1578953232.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql_query('select * from "messages"',con=conn)
/tmp/ipykernel_3838726/1578953232.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  channels = psql.read_sql_query('select * from "channels"',con=conn)
/tmp/ipykernel_3838726/1578953232.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = psql.read_sql_query('select * from "users"',con=conn)


In [3]:
df.sender_id = df.sender_id.astype('Int64')
df.reply_id = df.reply_id.astype('Int64')
df.forward_id = df.forward_id.astype('Int64')
df.sender_type = df.sender_type.astype('Int32')
df = df.drop(columns=['id', 'ts'])
df = pd.merge(df, channels[['channel_id', 'title']], on='channel_id', how='left').rename(columns={'title': 'channel_title'})
df = pd.merge(df, users[['user_id', 'first_name', 'last_name', 'username', 'phone']].rename(columns={'user_id': 'sender_id'}), on='sender_id', how='left')

In [4]:
df['short_action'] = df[df.action.notna()]['action'].str.extract(r'(^[^\(]*)\(')[0]
temp = df.loc[(df.short_action=='MessageActionChatAddUser')|(df.short_action=='MessageActionChatDeleteUser'), ['create_date', 'channel_id', 'channel_title', 'short_action']]
temp['user_count'] = 0
for i in temp['channel_id'].unique():
    j = 0
    for index, row in temp[temp['channel_id']==i].sort_values(by='create_date').iterrows():
      if row["short_action"] == 'MessageActionChatAddUser':
        j += 1
        temp.at[index, 'user_count'] = j
      if row["short_action"] == 'MessageActionChatDeleteUser':
        j -= 1
        temp.at[index, 'user_count'] = j
temp.head()

,create_date,channel_id,channel_title,short_action,user_count
35,2023-05-29 04:12:13+00:00,1417395612,Beregini Chat,MessageActionChatAddUser,1444
399,2023-05-29 13:50:51+00:00,1417395612,Beregini Chat,MessageActionChatAddUser,1445
428,2023-05-29 14:12:16+00:00,1417395612,Beregini Chat,MessageActionChatAddUser,1446
444,2023-05-29 14:36:12+00:00,1417395612,Beregini Chat,MessageActionChatAddUser,1447
458,2023-05-29 15:45:34+00:00,1417395612,Beregini Chat,MessageActionChatAddUser,1448


In [ ]:
temp.groupby(['channel_title']).agg({'user_count': 'max'}).reset_index().sort_values(by='user_count', ascending=False).reset_index(drop=True)

In [ ]:
temp.channel_title.unique()

In [ ]:
killnet = temp[temp.channel_title=='KILLNET ЧАТ 💎'].sort_values(by='create_date')
noname = temp[temp.channel_title=='NoName057(16) Chat'].sort_values(by='create_date')
killmilk = temp[temp.channel_title=='KillMilk Chat'].sort_values(by='create_date')
beregini = temp[temp.channel_title=='Beregini Chat'].sort_values(by='create_date')

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=killnet['create_date'], y=killnet['user_count'], name='killnet')
)
fig.add_trace(
    go.Scatter(x=noname['create_date'], y=noname['user_count'], name='noname')
)
fig.add_trace(
    go.Scatter(x=killmilk['create_date'], y=killmilk['user_count'], name='killmilk')
)
fig.add_trace(
    go.Scatter(x=beregini['create_date'], y=beregini['user_count'], name='beregini')
)
pio.renderers.default = 'iframe'
fig.show()

In [ ]:
df[df.action.isna()].groupby('channel_title').size().reset_index(name='count').sort_values(by='count', ascending=False).reset_index(drop=True).head(12)

In [ ]:
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)'
tt = df["text"].str.extract(pattern, expand=False).str.strip().reset_index()
tt[tt['text'].notna()]

In [ ]:
df[df.forward_id.notna()]

In [ ]:
df.groupby('forward_name').size().reset_index(name='count').sort_values(by='count', ascending=False).head(20)

In [ ]:
df[~df.file_extension.isin(['.tgs', '.ogg', '.png', '.oga', '.mp3', '.gif', '.m4v', '.mov', '.wav', '.m4a', '.mp4', '.webp', '.jpg'])].\
groupby(['filehash_sha1', 'file_extension']).size().reset_index(name='count').\
groupby('file_extension').size().reset_index(name='count').\
sort_values(by='count', ascending=False).reset_index(drop=True).head(20)

In [ ]:
df[ 
    (df.forward_id.isna()) & 
    (df['text'].str.contains('check-host.net', na=None, regex=False) == False) & 
    (df['text'].str.contains('https', na=None, regex=False))
]

In [ ]:
df[ 
    (df.forward_id.isna()) & (df.channel_id==1732250465) &
    (df['text'].str.contains('ddos', na=None, regex=False))
]

In [ ]:
df[ 
    (df.forward_id.isna()) & (df.channel_id==1732250465) &
    (
        (df['text'].str.contains('литовс', na=None, regex=False)) | 
        (df['text'].str.contains('литв', na=None, regex=False)) | 
        (df['text'].str.contains('Литовс', na=None, regex=False)) |
        (df['text'].str.contains('Литв', na=None, regex=False))
    )
].sort_values(by='create_date').to_csv("output.csv")

In [ ]:
df[(df.forward_id.notna())].groupby('sender_id').size().reset_index(name='count').sort_values(by='count', ascending=False).head(50)

In [ ]:
df[ 
    (df.forward_id.isna()) & (df.channel_id==1732250465) &
    (
        (df['text'].str.contains('литовс', na=None, regex=False)) | 
        (df['text'].str.contains('литв', na=None, regex=False)) | 
        (df['text'].str.contains('Литовс', na=None, regex=False)) |
        (df['text'].str.contains('Литв', na=None, regex=False))
    )
].sort_values(by='create_date')